<a href="https://colab.research.google.com/github/avaneeshp23/GenAIhackathon/blob/hackathon/genaihackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.3 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/avaneeshp23/GenAIhackathon.git

Cloning into 'GenAIhackathon'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 21 (delta 2), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 166.74 KiB | 828.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.


In [3]:
import os
os.environ['GROQ_API_KEY'] = "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"

In [4]:
from groq import Groq

In [5]:
client = Groq(api_key=os.environ['GROQ_API_KEY'])

In [6]:
import pandas as pd
import requests
import re

In [7]:
train_df = pd.read_csv('/content/GenAIhackathon/data/train.csv')

In [8]:
train_df.head()

,ticket_ID,ticket_subject,ticket_body,department,type,priority,language
0,1001,Discrepancia de facturación en Google Workspace,"Estimado equipo de soporte de TI,\n\nEstoy esc...",Billing and Payments,Incident,low,es
1,1002,Urgent Consultation Request for Critical IT Is...,"Dear IT Services Support Team, I hope this mes...",Customer Service,Request,high,en
2,1003,Consulta sobre Servicios de Consultoría en TI,"Estimado Servicio de Atención al Cliente,\n\nM...",General Inquiry,Request,medium,es
3,1004,Demande de mise à jour des dossiers,"Cher service client, \n\nJe vous écris pour de...",Human Resources,Change,low,fr
4,1005,Issues with Slack connection affecting team co...,"Dear Customer Support Team,\n\nI am encounteri...",Product Support,Problem,medium,en


In [9]:
train_df['text'] = train_df['ticket_subject'] + ' ' + train_df['ticket_body']
train_df = train_df[['text', 'department', 'type', 'priority', 'language']]

In [10]:
train_df.head()

,text,department,type,priority,language
0,Discrepancia de facturación en Google Workspac...,Billing and Payments,Incident,low,es
1,Urgent Consultation Request for Critical IT Is...,Customer Service,Request,high,en
2,Consulta sobre Servicios de Consultoría en TI ...,General Inquiry,Request,medium,es
3,Demande de mise à jour des dossiers Cher servi...,Human Resources,Change,low,fr
4,Issues with Slack connection affecting team co...,Product Support,Problem,medium,en


In [11]:
import os
api_key = os.getenv('GROQ_API_KEY')
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

In [12]:
data = {
    'model': 'deepseek-r1-distill-llama-70b',
    'train_data': train_df.to_dict(orient='records'),
    'task': 'classification',
    'labels': {
        'department': train_df['department'].unique().tolist(),
        'type': train_df['type'].unique().tolist(),
        'priority': train_df['priority'].unique().tolist(),
        'language': train_df['language'].unique().tolist()
    }
}

In [13]:
completion = client.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",
    messages=[
        {
            "role": "user",
            "content": "You are given a pandas dataframe train_df. You task is to automatically analyze and classify IT support tickets based on their content. Each ticket, consisting of a subject and body, should be accurately classified to predict the following attributes:\n\nDepartment: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry\nPriority: The urgency level such as low, medium, or high.\nLanguage: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following: https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html.\nType: The nature of the request, categorized as Incident, Request, Problem, or Change."
        }
    ],
    temperature=0.6,
    max_completion_tokens=4096,
    top_p=0.95,
    stream=True,
    stop=None,
)

In [14]:
def get_response(message, model="deepseek-r1-distill-llama-70b"):
    task_description = (
        "You are given a pandas dataframe row from train_df. Your task is to automatically analyze and classify IT support tickets based on their content. "
        "Each ticket, consisting of a Text, should be accurately classified to predict the following attributes:\n\n"
        "Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, "
        "Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry\n"
        "Priority: The urgency level such as low, medium, or high.\n"
        "Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following: "
        "https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html.\n"
        "Type: The nature of the request, categorized as Incident, Request, Problem, or Change."
    )
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": task_description},
            {"role": "user", "content": message}
        ],
        temperature=0.6,
        max_completion_tokens=4096,
        top_p=0.95,
        stream=False,
        stop=None,
    )
    return response.choices[0].message.content

In [15]:
results = []
for _, row in train_df.iterrows():
    message = row['text']
    response = get_response(message)
    results.append(response)

train_df['classification'] = results

In [16]:
train_df.head()

,text,department,type,priority,language,classification
0,Discrepancia de facturación en Google Workspac...,Billing and Payments,Incident,low,es,"<think>\nOkay, so I need to classify this IT s..."
1,Urgent Consultation Request for Critical IT Is...,Customer Service,Request,high,en,"<think>\nOkay, so I need to classify this IT s..."
2,Consulta sobre Servicios de Consultoría en TI ...,General Inquiry,Request,medium,es,"<think>\nOkay, so I've got this task where I n..."
3,Demande de mise à jour des dossiers Cher servi...,Human Resources,Change,low,fr,"<think>\nOkay, so I'm trying to figure out how..."
4,Issues with Slack connection affecting team co...,Product Support,Problem,medium,en,"<think>\nOkay, I need to analyze this IT suppo..."


In [17]:
test_df = pd.read_csv('/content/GenAIhackathon/data/test.csv')
test_df.head()

,ticket_ID,ticket_subject,ticket_body
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...


In [18]:
test_df['text'] = test_df['ticket_subject'].fillna('') + ' ' + test_df['ticket_body'].fillna('')
test_df.shape

(500, 4)

In [19]:
small_df = test_df[:5]
small_df.head()

,ticket_ID,ticket_subject,ticket_body,text
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand...",Urgente: problema con la funzione di condivisi...
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe...",Abrechnungsdiskrepanzen in Google Workspace Li...
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr...",Problèmes avec la fonction sans fil de l'Epson...
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques...","Gentile assistenza clienti, \n\nSpero che que..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...,Assistenza necessaria per gli arresti anomali ...


In [ ]:
results = []
for _, row in test_df.iterrows():
    message = row['text']
    response = get_response(message)
    #print("Response:", response)
    # Extract the classifications from the response
    classifications = {
        'department': None,
        'priority': None,
        'language': None,
        'type': None
    }
    department_match = re.search(r"Department:\s*(.*?)(\n|$)", response)
    priority_match = re.search(r"Priority:\s*(.*?)(\n|$)", response)
    language_match = re.search(r"Language:\s*(.*?)(\n|$)", response)
    type_match = re.search(r"Type:\s*(.*?)(\n|$)", response)

    if department_match:
        classifications['department'] = re.split(r'[\n.,]', department_match.group(1))[0].replace('**', '').strip()
    if priority_match:
        classifications['priority'] = re.split(r'[\n.,]', priority_match.group(1))[0].replace('**', '').strip()
    if language_match:
        classifications['language'] = re.split(r'[\n.,]', language_match.group(1))[0].replace('**', '').strip()
        # Remove any additional text in parentheses
        classifications['language'] = re.sub(r'\s*\(.*?\)', '', classifications['language'])
    if type_match:
        classifications['type'] = re.split(r'[\n.,]', type_match.group(1))[0].replace('**', '').strip()

    # Handle cases where the response format is different
    if not classifications['department']:
        department_match = re.search(r"Department:\s*(.*?)(\n|$)", response)
        if department_match:
            classifications['department'] = re.split(r'[\n.,]', department_match.group(1))[0].strip()
    if not classifications['priority']:
        priority_match = re.search(r"Priority:\s*(.*?)(\n|$)", response)
        if priority_match:
            classifications['priority'] = re.split(r'[\n.,]', priority_match.group(1))[0].strip()
    if not classifications['language']:
        language_match = re.search(r"Language:\s*(.*?)(\n|$)", response)
        if language_match:
            classifications['language'] = re.split(r'[\n.,]', language_match.group(1))[0].strip()
    if not classifications['type']:
        type_match = re.search(r"Type:\s*(.*?)(\n|$)", response)
        if type_match:
            classifications['type'] = re.split(r'[\n.,]', type_match.group(1))[0].strip()

    results.append(classifications)


In [89]:
results

[{'department': 'IT Support',
  'priority': 'High',
  'language': 'it',
  'type': 'Incident'},
 {'department': 'Billing and Payments',
  'priority': 'Medium',
  'language': 'de',
  'type': 'Incident'},
 {'department': 'Product Support',
  'priority': 'Medium',
  'language': 'fr',
  'type': 'Problem'},
 {'department': 'Billing and Payments',
  'priority': 'Medium',
  'language': 'it',
  'type': 'Request'},
 {'department': 'Technical Support',
  'priority': 'High',
  'language': 'it',
  'type': 'Incident'}]

In [90]:
output_df = pd.DataFrame(results)
output_df['ticket_ID'] = test_df.head(5)['ticket_ID']
output_df = output_df[['ticket_ID', 'department', 'type', 'priority', 'language']]

In [91]:
output_df.head()

,ticket_ID,department,type,priority,language
0,1,IT Support,Incident,High,it
1,2,Billing and Payments,Incident,Medium,de
2,3,Product Support,Problem,Medium,fr
3,4,Billing and Payments,Request,Medium,it
4,5,Technical Support,Incident,High,it
